In [251]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize, LabelBinarizer
import sys
sys.path.append('../..')
from modules.many_features import utils, constants

In [252]:
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

#### The Datasets

In [286]:
#train_df = pd.read_csv('../../final/data/train_set_noisy_6_missing_3.csv')
train_df = pd.read_csv('../../../anemia_ml4hc/data/train_set_noisiness_0.2_missingness_0.4.csv')
train_df = train_df.fillna(-1)
# train_df = train_df.replace(-1, np.nan)
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
# X_cols = X_train.columns
X_train.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat
0,12.570316,-1.000000,4.347154,2.195594,274.677370,98.593975,-1.000000,-1.000000,1,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,37.710949,-1.000000
1,6.025877,65.947255,5.269617,-1.000000,-1.000000,78.152338,-1.000000,-1.000000,0,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
2,7.971342,-1.000000,3.031102,0.000000,391.557192,102.908562,-1.000000,2.323813,1,-1.000000,78.906129,-1.0,-1.000000,6.731272,-1.000000,23.914025,-1.000000
3,8.648926,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.674076,0,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
4,16.785024,410.993164,3.460755,-1.000000,-1.000000,-1.000000,143.401682,6.284396,1,0.290346,89.925567,-1.0,33.165717,-1.000000,127.932614,-1.000000,56.439819


In [287]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]
X_test.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000


In [288]:
# X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((50400, 17), (14000, 17), (50400,), (14000,))

In [289]:
y_test

array([5, 1, 4, ..., 2, 4, 6])

#### Normalizing X

In [290]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [291]:
mmc = MinMaxScaler()
# mmc = StandardScaler()

In [292]:
X_train_norm = mmc.fit_transform(X_train)
X_test_norm = mmc.transform(X_test)

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



In [293]:
X_train_norm.shape, X_test_norm.shape

((50400, 17), (14000, 17))

#### Support Vector Machine

In [294]:
from sklearn.svm import SVC
svm_model = SVC(kernel='rbf', C=1, decision_function_shape='ovo', random_state=constants.SEED).fit(X_train_norm, y_train)
y_pred_svm = svm_model.predict(X_test_norm)
test_df_svm = pd.DataFrame()
test_df_svm['y_actual'] = y_test
test_df_svm['y_pred'] = y_pred_svm
test_df_svm.isna().sum()

y_actual    0
y_pred      0
dtype: int64

In [295]:
success_rate_svm, success_df_svm = utils.success_rate(test_df_svm)
success_rate_svm

76.27142857142857

In [296]:
acc_svm, f1_svm, roc_auc_svm = utils.test(test_df_svm['y_actual'], test_df_svm['y_pred'])
acc_svm, f1_svm, roc_auc_svm

(76.27142857142857, 70.454037499695, 85.16825821233365)

#### Feed-forward Neural Network - Keras

In [182]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras import backend as K
import keras

ModuleNotFoundError: No module named 'keras'

In [ ]:
#keras.utils.set_random_seed(42)

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
def numerize_labels(pd_series):
    series_copy = pd_series.copy()
    series_copy = series_copy.map(class_dict)
    return series_copy

y_train_num = y_train
y_test_num = y_test

dummy_y_train = np_utils.to_categorical(y_train_num)
dummy_y_test = np_utils.to_categorical(y_test_num)
dummy_y_train.shape, dummy_y_test.shape

In [ ]:
model = Sequential()
model.add(Dropout(0.5, input_shape=(17,)))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', f1_m, precision_m, recall_m])
history = model.fit(X_train_norm, dummy_y_train, epochs=100, batch_size=1024, shuffle=True, validation_split=0.20, verbose=0)
#model.save('../models/baselines/synthentic_nn.h5')
loss, accuracy, f1, precision, recall = model.evaluate(X_test_norm, dummy_y_test, verbose=0)
loss, accuracy, f1, precision, recall

In [ ]:
#Alternatively
y_pred1_nn = model.predict(X_test_norm)
y_pred_nn = np.argmax(y_pred1_nn, axis=1)
np.unique(y_pred_nn)

In [ ]:
test_df_nn = pd.DataFrame()
test_df_nn['y_actual'] = y_test
test_df_nn['y_pred'] = y_pred_nn
test_df_nn.isna().sum()

In [ ]:
success_rate_nn, success_df_nn = utils.success_rate(test_df_nn)
success_rate_nn

In [ ]:
acc_nn, f1_nn, roc_auc_nn = utils.test(test_df_nn['y_actual'], test_df_nn['y_pred'])
acc_nn, f1_nn, roc_auc_nn